https://jira.x5.ru/browse/CVMXC-1957

In [1]:
name = 'CVMXC-1957'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='bdse', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

In [7]:
seg_no = 'seg1'

### Проверка на доступность плю в магазинах

Северо-Запад, Москва

In [8]:
macroregion_dk = ['MRDND', 'MRDCT']

In [9]:
plu_codes = [3472773]

In [10]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            #.filter(F.col('federal_subject_dk').isin([23, 93, 123, 193, 74, 174,
             #                                         774, 16, 116, 716, 71, 32]))
            .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

5048

### Выбираем гостей нужного юзкейса

In [14]:
usecase = ['cross', 'upgrade', 'ump']
lifetime = 90
freq = 1
dt = datetime.date.today()
# plu_hierarchy_lvl_4_dk = 'FD0910005'
syntethic_category_id = [65]
plu_brand_code = ['2572','3403','O899','2268','1120','2325','0844','2251']
base_qty_per_period = 1
plu_netto_weight_amt = 0.5

In [15]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [16]:
start_date = datetime.date(2021, 8, 10)
end_date = datetime.date(2021, 11, 9)

In [17]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [18]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [19]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(customers_usecase, on=['account_no', 'customer_rk'], how='inner')

In [20]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [21]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'base_qty' #count
                               )
                   )


In [22]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_netto_weight_amt') >= plu_netto_weight_amt)
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [23]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [24]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [25]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= base_qty_per_period)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [26]:
seg = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [27]:
seg.write.parquet('temp', mode='overwrite')
seg = spark.read.parquet('temp')

### Проверяем на доступность отобранных гостей в определенную дату

In [28]:
check_date = '2021-12-06'

In [29]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:25


In [30]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:11


In [31]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:10


In [32]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [34]:
seg_sms.write.parquet('temp01', mode='overwrite')
seg_sms = spark.read.parquet('temp01')

In [35]:
segm = (seg.join(seg_sms, on='customer_rk', how='inner')
                .select('customer_rk')
                .distinct())

In [36]:
seg_sms_pd = segm.toPandas()

In [37]:
len(seg_sms_pd)

45372

In [38]:
seg_sms_pd.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [39]:
seg_sms_pd = seg_sms_pd.astype('str')

In [40]:
cross = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('1')) | (seg_sms_pd['customer_rk'].str.endswith('2'))
              | (seg_sms_pd['customer_rk'].str.endswith('3')) | (seg_sms_pd['customer_rk'].str.endswith('4'))].reset_index(drop=True).copy()

In [41]:
upgrade = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('5')) | (seg_sms_pd['customer_rk'].str.endswith('6'))
              | (seg_sms_pd['customer_rk'].str.endswith('7')) | (seg_sms_pd['customer_rk'].str.endswith('8'))].reset_index(drop=True).copy()

In [42]:
ump = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('0'))
              | (seg_sms_pd['customer_rk'].str.endswith('9'))].reset_index(drop=True).copy()

In [43]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [44]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [45]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [46]:
print('cross: {}, upgrade: {}, ump: {}'.format(len(cross), len(upgrade), len(ump)))

cross: 14828, upgrade: 20326, ump: 10218


# seg2

In [47]:
seg_no = 'seg2'

### Проверка на доступность плю в магазинах

Северо-Запад, Москва

In [48]:
macroregion_dk = ['MRDND', 'MRDCT']

In [49]:
plu_codes = [3472773]

In [50]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            #.filter(F.col('federal_subject_dk').isin([23, 93, 123, 193, 74, 174,
             #                                         774, 16, 116, 716, 71, 32]))
            .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

5048

### Выбираем гостей нужного юзкейса

In [51]:
usecase = ['cross', 'upgrade', 'ump']
lifetime = 90
freq = 1
dt = datetime.date.today()
# plu_hierarchy_lvl_4_dk = 'FD0910005'
syntethic_category_id = [65]
plu_brand_code = ['2572','3403','2268','1120','2325','0844']
base_qty_per_period = 1
plu_netto_weight_amt_min = 0.25
plu_netto_weight_amt_max = 0.45

In [52]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [53]:
start_date = datetime.date(2021, 8, 10)
end_date = datetime.date(2021, 11, 9)

In [54]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [55]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [56]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(customers_usecase, on=['account_no', 'customer_rk'], how='inner')

In [57]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [58]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'base_qty' #count
                               )
                   )


In [60]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_netto_weight_amt').between(plu_netto_weight_amt_min, plu_netto_weight_amt_max))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [61]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [62]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [63]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= base_qty_per_period)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [64]:
seg = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [65]:
seg.write.parquet('temp', mode='overwrite')
seg = spark.read.parquet('temp')

### Проверяем на доступность отобранных гостей в определенную дату

In [66]:
check_date = '2021-12-06'

In [67]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [68]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [69]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=seg, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [70]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [71]:
seg_sms.write.parquet('temp01', mode='overwrite')
seg_sms = spark.read.parquet('temp01')

In [72]:
segm = (seg.join(seg_sms, on='customer_rk', how='inner')
                .select('customer_rk')
                .distinct())

In [73]:
seg_sms_pd = segm.toPandas()

In [74]:
len(seg_sms_pd)

139980

In [75]:
seg_sms_pd.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [76]:
seg_sms_pd = seg_sms_pd.astype('str')

In [77]:
cross = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('1')) | (seg_sms_pd['customer_rk'].str.endswith('2'))
              | (seg_sms_pd['customer_rk'].str.endswith('3')) | (seg_sms_pd['customer_rk'].str.endswith('4'))].reset_index(drop=True).copy()

In [78]:
upgrade = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('5')) | (seg_sms_pd['customer_rk'].str.endswith('6'))
              | (seg_sms_pd['customer_rk'].str.endswith('7')) | (seg_sms_pd['customer_rk'].str.endswith('8'))].reset_index(drop=True).copy()

In [79]:
ump = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('0'))
              | (seg_sms_pd['customer_rk'].str.endswith('9'))].reset_index(drop=True).copy()

In [80]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [81]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [82]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [83]:
print('cross: {}, upgrade: {}, ump: {}'.format(len(cross), len(upgrade), len(ump)))

cross: 46789, upgrade: 61892, ump: 31299


21/11/16 06:11:15 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
21/11/16 06:11:15 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Attempt recovered after RM restartDue to executor failures all available nodes are blacklisted
21/11/16 06:11:15 ERROR client.TransportClient: Failed to send RPC RPC 5238084258498323121 to /192.168.234.60:44176: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
21/11/16 06:11:15 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(0,0,Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 5238084258498323121 to /192.168.234.60:44176: java.nio.channels.ClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.jav